In [ ]:
!pip install -q langchain transformers PyPDF2 sentence-transformers install pinecone-client xformers bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 19.4 

In [ ]:
import pandas as pd
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain import HuggingFacePipeline, PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import os
import pinecone

In [ ]:
#mounting Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Read in HP troubleshooting pdf
hp_guide=PdfReader('/content/gdrive/MyDrive/Colab Notebooks/Langchain/HP_project/HP_troubleshooting.pdf')

In [ ]:
#Extract all of the text from the pdf
hp_guide_text=''
for i, page in enumerate(hp_guide.pages):
  text=page.extract_text()
  if text:
    hp_guide_text+=text

hp_guide_text

In [ ]:
#Split the text into chunks for use in ChatGPT
text_splitter=RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=200)
split_text=text_splitter.split_text(hp_guide_text)
print("Number of text chunks: {}  \n".format(len(split_text)))

print("Example of text_chunk \n")
split_text[0]


Number of text chunks: 284  

Example of text_chunk 



'Troubleshooting Guide www.hp.com 1–11\nComputer Diagnostic Features\nDiagnostics for Windows\nThe Diagnostics for Windows (DFW)  utility allows you to view \ninformation about the hardware and software configuration of the computer while running Microsoft Windows XP. It also allows you to perform hardware and software te sts on the subsystems of the \ncomputer.\nWhen you invoke Diagnostics for Wi ndows, the Overview screen is'

In [ ]:
#Instantiating Sentence transformers embeddings to use for Pinecone index
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
#initializing pinecone environement
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', 'removed')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'removed')
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)



In [ ]:
#creating Pinecone index - Commented out so that index is not recreated with each notebook run
#hp_embeddings_indexed=Pinecone.from_texts(split_text, embeddings, index_name="hptroubleshooting")

In [ ]:
#downloading already created index from Pinecone
hp_embeddings_indexed = Pinecone.from_existing_index("hptroubleshooting", embeddings)

##Downloading LLama 2 Model/Tokenizer and creating a pipeline

In [ ]:
#Downloading optimizer
tokenizer= AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
#Downloading the Llama-2 model
model=AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf",
                                           torch_dtype=torch.float16,
                                           load_in_8bit=True)

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
#creating Hugging face pipeline
pipeline=pipeline("text-generation",
                  model=model,
                  tokenizer=tokenizer,
                  torch_dtype=torch.float16,
                  max_new_tokens=512,
                  do_sample=True,
                  top_p=0.3

                  )

In [ ]:
#creating Langchain Wrapper for Hugging Face pipeline
llm = HuggingFacePipeline(
    pipeline=pipeline,
    model_kwargs={"temperature": 0.1}
)

##LangChain Question and Answer Retrieval From Pinecone Indexed Document

In [ ]:
# System prompt with a message to not make up answers
system_prompt = """You are a helpful and honest assistant. Use the context provided to answer the question.
If you don't know the answer, just say that you don't know. Do not make up an answer. Do not repeat yourself over and over"""

#User question and reference to the retrieved information
info_and_user_message = """
Context: {context}

Question: {question}
"""
#Prompt template formatted to Llama 2 specification
llama_prompt_template=f"<s> [INST] <<SYS>>\n {system_prompt} \n<</SYS>>\n\n {info_and_user_message } [/INST]"

In [ ]:
#Langchian prompt template with prompt above
prompt = PromptTemplate(template=llama_prompt_template, input_variables=["context", "question"])

In [ ]:
#Q/A retrieval chain with prompt above and retriever to get the best 3 matches from the Pinecone database index
retrieval_chain = RetrievalQA.from_chain_type(llm=llm,
                                            chain_type="stuff",
                                            retriever=hp_embeddings_indexed.as_retriever(search_kwargs={"k":3}),

                                            chain_type_kwargs={"prompt": prompt}

                                            )

In [ ]:
#function to print output
def print_output(output):
    # Print the query
    print("Query: {}\n".format(output['query']))
    output["result"]=output['result'].replace("\\n\n", " ")
    words = output["result"].split()
    lines = [' '.join(words[i:i+25]) for i in range(0, len(words), 25)]
    # Print the result
    print("Result: {} ".format('\n'.join(lines)))

In [ ]:
#Question 1
result = retrieval_chain("Hard drive boot has been disabled in Computer Setup. What is the solution?")
print_output(result)

Query: Hard drive boot has been disabled in Computer Setup. What is the solution?

Result: Based on the provided context, the solution to the problem "Hard drive boot has been disabled in Computer Setup" is: Run Computer Setup and enable
the hard drive entry in the Storage > Boot Order list. This is because the hard drive is not listed as the first bootable device
in the Boot Order list, which is causing the system to fail to boot from the hard drive. By enabling the hard drive entry in
the Boot Order list, the system should be able to boot from the hard drive successfully. 


In [ ]:
#Question 2
result_2 = retrieval_chain("What is the purpose of the CMOS and how is it reset?")

In [ ]:
print_output(result_2)

Query: What is the purpose of the CMOS and how is it reset?

Result: The purpose of the CMOS (Configuration Memory) is to store password information and configuration settings for the computer. The CMOS button is used to clear
and reset the CMOS, which will erase any customized information, including passwords, asset numbers, and special settings. To reset the CMOS, the power cord must
be disconnected from the wall outlet, and the CMOS button must be pressed and held for five seconds. It is important to back up the
CMOS settings before resetting them in case they are needed later. The Computer Setup utility can be used to reset the CMOS through the computer's
setup menu. 


In [ ]:
#Question 3
result_3 = retrieval_chain("How do you install Diagnostics for Windows on Windows XP?")

In [ ]:
print_output(result_3)

Query: How do you install Diagnostics for Windows on Windows XP?

Result: Sure, I'd be happy to help! According to the context provided, to install Diagnostics for Windows on Windows XP, you should follow these steps: 1.
Close any Windows applications. 2. Install the Diagnostics for Windows utility: * In Windows XP, select Start > Software Setup icon. * Select Diagnostics for
Windows > Next button, then follow the instructions on the screen. * If the Setup Software icon is not on the Desktop or in the
Start menu, run the Setup program from the C:\CPQAPPS\DIAGS directory or select Start > Run and type the following in the command line: C:\CPQAPPS\DIAGS\SETUP. 3.
Click Next to install Diagnostics for Windows. 4. After the program has finished installing, you may be prompted to install the program. If you want
to upgrade an existing version of Diagnostics for Windows installed on the computer, you can visit the HP website to locate and download the latest
Diagnostics for Windows SoftPaq. 